## Copyright
Copyright (c) 2021, Her Majesty in Right of Canada as represented by the National Research Council Canada. Rights provided under GNU GENERAL PUBLIC LICENSE, Version 3. Full text of the license accessible at the [LICENSE](LICENSE) file.

---

Los datos se recopilan automaticamente de las siguientes fuentes, se procesan, seleeccionan y almacenan directamente en el dispositivo.

* ButterflyNetwork
* GrepMed
* LITFL
* The PocusAtlas
* Radiopaedia
* CoreUltrasound
* University of Florida (UF)
* Scientific Publications
* Clarius

También se extraen imagenes a partir de los videos recopilados, de igual manera se procesan, seleccionan y almacenan en el dispositivo.

__Nota:__ No se almacenan datos en el repositorio NRC-COVIDx-US y solo contiene scripts para recopilar, seleccionar e integrar datos sistematicamente en el dispositivo.

---

#### Librerias

In [1]:
import pandas as pd #Manipulación y analisis de datos
import numpy as np #Calculos numericos y manejo de arreglos
import os #Creación y eliminación de directorios
import re #Limpiar nombres de archivos y extrae información de cadenas
import shutil #Copiar y mover archivos - organizar imagenes
import random #Dividir datos en conjuntos de entrenamiento y prueba - aleatoriamente

import cv2 #Para procesamiento de imagenes - Cargar, transformar y realizar operaciones de preprocesamiento
from PIL import Image #Complementa a OpenCV - operaciones basicas de manipulación de imágenes 

import zipfile #Permite trabajar con arhivos Zip - descomprimirlos
from selenium import webdriver #Automatizar navegadores web - descargar imágenes
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests #Hacer solicitudes HTTP - para descarga de imágenes
from vimeo_downloader import Vimeo # Descargar videos de Vimeo - análisis de imágenes en movimiento 
import urllib.request # Opción para solicitudes HTTP - complementa a requests
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 

from progressbar import ProgressBar # Mostrar barras de progreso en la consola - para descargas o procesamientos

import time # Medir rendimiento del codigo o implementar pausas
#from image_data import extract_images # Modulo personalizado - contiene la función de extracción de imágenes

import matplotlib.pyplot as plt # Visualización de datos - mostrar imágenes y gráficos de rendimiento

import subprocess # Ejecutar comandos del sistema 
import glob # Facilita busqueda de archivos mediante patrones - util para cargar imágenes de un directorio 



In [2]:
# Se imprimen las versiones de las librerias - Pandas, Selenium y Requests
print("Pandas", pd.__version__)
import selenium
print("selenium", selenium.__version__)
print("requests", requests.__version__)

Pandas 2.2.3
selenium 4.25.0
requests 2.32.3


#### Funciones

In [3]:
def get_download_path():
    """Devuelve la ruta de descarga predeterminada de Windows o Linux"""
    if os.name == 'nt': #Sistema Operativo Windows
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        ####  Control de errores
        try:
            with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
                location = winreg.QueryValueEx(key, downloads_guid)[0]
            return location
        except FileNotFoundError:
            print("No se encontró la clave de registro para la carpeta de descargas")
            return None
    else: #Sistema Operativo Linux
        return os.path.join(os.path.expanduser('~'), 'downloads')   

def remove_html_tags(text):
    """Función para eliminar etiquetas html en cadena"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


In [4]:
# Código para descargar archivos zip de google drive, en caso de ser necesario
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    #Manejo de errores
    try:
        response = session.get(URL, params = { 'id' : id }, stream = True)
        print(f"Accediendo a: {response.url}")
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error intentando conectarse con Google Drive: {e}")
        return False
    
    token = get_confirm_token(response)

    if token:
        response = session.get(URL, params = { 'id' : id, 'confirm' : token }, stream = True)

    #Verificación de la respuesta ...
    if response.status_code == 200:
        save_response_content(response, destination)
        print("Descarga completa")
        return True
    else:
        print(f"Error en la descarga: {response.status_code}")
        return False

        

    save_response_content(response, destination)    

#Busca advertencias de Google Drive, indica si sera necesaria una confirmación extra para la descarga de un archivo 
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

#Guarda el contenido de la respuesta en el archivo destino, descargando en chunks
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    progress = ProgressBar() 
    
    with open(destination, "wb") as f:
        for chunk in progress(response.iter_content(CHUNK_SIZE)):
            if chunk: # Filtrar nuevos chunks de salida que se mantengan-activos
                f.write(chunk)

#### Parámetros

In [5]:
# Establecer un directorio de ruta de guardado
SAVE_PATH = 'data' #Indica el directorio de guardado

# Crear carpetas de datos, videos e imagenes, en el caso de que no existan
directories = ['data', 'data/video', 'data/image']
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
    
# Configuración del ChromeDriver
if os.name == 'nt': # Windows
    chromedriver = "utils/chromedriver.exe" 
else: #Linux
    chromedriver = "utils/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver #Define el path a chromedriver.exe
chrome_options = Options()
chrome_options.add_argument("--headless") #Asegura la ejecución de Chrome en modo Headless (Sin abrir una ventana grafica)

# Configuración de variables globales - Son variables que apuntan a las carpetas de videos e imagenes respectivamente
VIDEO_PATH = 'data/video/'
IMAGE_PATH = 'data/image/'

#### Leer los Archivos de Metadatos

In [6]:
#Lee un archivo CSV que contiene metadatos de video 
try:
    metadata = pd.read_csv('utils/video_metadata.csv', sep=',', encoding='latin1')
except FileNotFoundError:
    print("Error: No se encuentra el archivo 'video_metadata.csv'. ")
except pd.errors.ParserError:
    print("Error: El archivo 'video_metadate.csv' tiene un formato incorrecto. ")
print(metadata.shape)
metadata.head(2)

(244, 21)


,id,filename,filetype,folder,source,url,probe,class,class_on_website,version,...,type,patient,case_no,gender,age,comment,paper_link,paper_doi,license,link
0,1_butterfly_covid,Coalescing B lines.mp4,mp4,data\tmp\Butterfly\B lines,Butterfly,https://butterflynetwork.getbynder.com/transfe...,Convex,COVID,NaN,1.0,...,lung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_butterfly_covid,Confluent B lines.mp4,mp4,data\tmp\Butterfly\B lines,Butterfly,https://butterflynetwork.getbynder.com/transfe...,Convex,COVID,NaN,1.0,...,lung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. Obtención de Videos de Ultrasonido

## 1.1. ButterflyNetwork

__Nota 1:__ Leer el archivo de metadatos, dependiendo de la configuración del sistema, puede que el boton de descarga no se muestre y si se llega a recibir un error, se puede aumentar el tiempo de suspension con el siguiente codigo...

```python
time.sleep(5)
```

__Nota 2:__ El siquiente bloque de codigo funciona con el navegador web "Chrome" y con la version 88 de ChromeDriver-incluido en la carpeta utils. Y puede haber algun error de la version de ChromeDriver se deba a la version que se posee del navegador Chrome. Si llega a ocurrir, se debe descargar la version correcta de ChromeDriver segun su version en el siguiente enlace (https://chromedriver.chromium.org/downloads) - Copiarla en la carpeta Utils

In [12]:
## URL del archivo Zip
#butterfly_url = metadata[(metadata.source == 'Butterfly') & (metadata.date_added == 'Mar_2021')].url.unique()[0]
#print('... Descargando archivo Zip de ButterflyNetwork ...')
#print(butterfly_url)
##Inicializar chrome_options
#chrome_options = Options()
#chrome_options.add_argument("__headless")

#browser=None #Inicializar browser

#try:
#    # Definir la ruta de Chrome driver 
#    service = Service("utils/chromedriver.exe")
#    browser = webdriver.Chrome(service=service, options=chrome_options)
    
#    #Funcón para esperar hasta que el botón sea interactuable
#    browser.get(butterfly_url)
#    WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn-primary'))).click()
    
#    # Ruta del archivo Zip descargado
#    zip_file_path = os.path.join(get_download_path(), 'Published -20210112T164653Z-001.zip')  # Nueva version - checked March 9, 2021

#    # Esperar hasta que se descargue el archivo Zip 
#    while not os.path.exists(zip_file_path):
#        time.sleep(1)

#    # Crear una carpeta Butterfly debajo de la carpeta de videos, solo si no existe
#    if not os.path.exists('data/tmp/Butterfly'):
#        os.makedirs('data/tmp/Butterfly')
#    time.sleep(2)

#    print('...Extrayendo archivos de video...')

#    # Extraer el archivo Zip descargado y se elimina el archivo zip tras completar la extracción
#    os.rename(zip_file_path, zip_file_path.replace(' ', ''))
#    process=subprocess.Popen("utils/7z.exe" +' x ' + zip_file_path.replace(' ', '') + ' -o' + 'data/tmp/Butterfly',stdout=subprocess.PIPE)
#    process.wait()
    
#    # Copiar archivos de subcarpetas a la carpeta de video - copia los archivos que no son imagenes a las carpetas de videos
#    for root, dirs, files in os.walk('data/tmp/Butterfly/Published_'):  
#        for file in files:
#            if file.endswith(".png"):
#                continue
#            path_file = os.path.join(root,file)
#            shutil.copy2(path_file, 'data/video') 

#    # Renombrar los archivos extraidos a sus identificadores
#    progress = ProgressBar() 
#    for root, dirs, files in os.walk('data/video'):  
#        for file in progress(files):
#            if file.endswith(".png"):
#                continue
#            path_file = os.path.join(root,file)
#            file_id = metadata[metadata.filename == file].id.values[0] + '.mp4'
#            # Renombrar el archivo a su id
#            os.rename(path_file, os.path.join(root,file_id))

#    print('=== Extracción de archivos de video ButterflyNetwork exitosa! ===')        
        
#    # Eliminar la carpeta tmp y su contenido
#    shutil.rmtree('data/tmp')

#    # Eliminar el archivo Zip
#    os.remove(zip_file_path.replace(' ', ''))
#except Exception as e:
#    print(f"Ocurrio un error durante el proceso de descarga: {e}")
#finally:
#    if browser: #Verifica si browser se inicializo
#        browser.quit()

**Nota :** Si se tiene un error al ejecutar la celda anterior, la siguiente celda se debe descomentar y ejecutar la siguiente celda en su lugar

In [48]:
import requests

print('Descargando archivos de video de ButterflyNetwork en formato Zip')
file_id = '18I4N6lWdcUW618Qwr6Krsd1Rkn946Ag0' # ID de enlace para drive

zip_file_path = os.path.join(get_download_path(), 'butterfly.zip')
download_file_from_google_drive(file_id, zip_file_path)  #Descarga del archivo con la función antes definida

#Verificar estado del archivo descargado
if os.path.exists(zip_file_path):
    print('Download complete\nExtracting video files')
    try:
        #Descomprimir archivos Zip 
        open_file = subprocess.Popen("utils/7z.exe" +' x ' + zip_file_path + ' -o' + 'data/video',stdout=subprocess.PIPE)
        stdout, stderr = open_file.communicate() # Ejecuta y captura la salida y errores de un proceso
        if open_file.returncode == 0:
            print('Extracción de archivos completa')
        else:
            print(f"Error durante el proceso de extracción: {stderr.decode()}")
    except Exception as e:
        print(f"Error durante la extracción: {e}")
else:
    print("El archivo no esta disponible para la descompresión")    

Descargando archivos de video de ButterflyNetwork en formato Zip


/ |#                                                  | 0 Elapsed Time: 0:00:00
| |#                                                  | 0 Elapsed Time: 0:00:00


Accediendo a: https://drive.usercontent.google.com/download?id=18I4N6lWdcUW618Qwr6Krsd1Rkn946Ag0&export=download
Descarga completa
Download complete
Extracting video files
Error durante la extracción: 'NoneType' object has no attribute 'decode'


## 1.2. GrepMed

In [49]:
print('...Extrayendo archivos de video de GrepMed...')

NumDes = 0
NumT = 0

#Se seleccionan las filas de "metadata" donde la fuente es GrepMed
grepmed_df = metadata[metadata.source == 'GrepMed']

#Avance de descarga
progress = ProgressBar(max_value=grepmed_df.shape[0]) 

for idx, row in progress(grepmed_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    try:
        #Descarga el archivo de video desde la URL
        vid = requests.get(row.url).content
        #Guarda el archivo en el directorio que se especifica
        with open(os.path.join('data/video/', filename), 'wb') as handler:
            handler.write(vid)
            NumDes = NumDes + 1
            print(f'=== Archivo de video {filename} extraido correctamente! ===')    
    except Exception as e:
        print(f"Error durante la descarga del archivo:{filename} {e}")   

if NumDes == NumT:
    print("=== Todos los archivos de video de GrepMed se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")
    



  0% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo archivos de video de GrepMed...


  5% (1 of 20) |#                        | Elapsed Time: 0:00:00 ETA:   0:00:11


=== Archivo de video 23_grepmed_pneumonia.mp4 extraido correctamente! ===


 10% (2 of 20) |##                       | Elapsed Time: 0:00:01 ETA:   0:00:10


...

=== Archivo de video 40_grepmed_pneumonia.mp4 extraido correctamente! ===


 95% (19 of 20) |######################  | Elapsed Time: 0:00:09 ETA:   0:00:00


=== Archivo de video 41_grepmed_pneumonia.mp4 extraido correctamente! ===


100% (20 of 20) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


=== Archivo de video 42_grepmed_covid.mp4 extraido correctamente! ===
=== Todos los archivos de video de GrepMed se descargaron correctamente ===
 20 archivos descargados


## 1.3. LITFL

In [50]:
print('...Extrayendo los archivos de video de LITFL...')
litfl_df = metadata[metadata.source == 'Litfl'] #Seleccionar fila de metadatos con fuente LITFL
NumDes = 0
NumT = 0

progress = ProgressBar(max_value=litfl_df.shape[0]) 
for idx, row in progress(litfl_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    
    # Descarga el archivo de vídeo desde "row.url"
    try:
        #Solicitud con tiempo de espera de 10 segundos 
        vid = requests.get(row.url, timeout=10) #.content
        vid.raise_for_status() #Verificar posibles errores
        #Guarda el archivo en el disco
        with open(os.path.join('data/video/', filename), 'wb') as handler:
            handler.write(vid.content)
            print(f'=== Archivo {filename} descargado y guardado correctamente! ===')    
            NumDes = NumDes + 1    
        time.sleep(1) #tiempo de espera adicional para evitar sobrecargas
    except requests.exceptions.Timeout:
        print(f"Error: El tiempo de espera se agoto mientras se intentaba la descarga {filename}.")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar {filename}: {e}")
    except Exception as e:
        print(f"Error inesperado en {filename}: {e}")

if NumDes == NumT:
    print("=== Todos los archivos de video de LITFL se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")
    

  0% (0 of 63) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo los archivos de video de LITFL...
=== Archivo 43_litfl_pneumonia.mp4 descargado y guardado correctamente! ===


  1% (1 of 63) |                         | Elapsed Time: 0:00:12 ETA:   0:13:01


=== Archivo 44_litfl_pneumonia.mp4 descargado y guardado correctamente! ===


  3% (2 of 63) |                         | Elapsed Time: 0:00:13 ETA:   0:07:03


...

=== Archivo 150_litfl_other.mp4 descargado y guardado correctamente! ===


 98% (62 of 63) |####################### | Elapsed Time: 0:01:38 ETA:   0:00:01


=== Archivo 151_litfl_pneumonia.mp4 descargado y guardado correctamente! ===


100% (63 of 63) |########################| Elapsed Time: 0:01:39 Time:  0:01:39


=== Todos los archivos de video de LITFL se descargaron correctamente ===
 63 archivos descargados


## 1.4. The POCUS Atlas

In [51]:
print('...Extrayendo archivos de video de The POCUS Atlas...')
pocus_df = metadata[metadata.source == 'PocusAtlas']
progress = ProgressBar(max_value=pocus_df.shape[0]) #Barra de progreso
NumDes = 0
NumT = 0

for idx, row in progress(pocus_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    try:
        #Solicitud para descarga de archivos
        vid = requests.get(row.url, timeout=10) #.content
        vid.raise_for_status() #Verificación de errores en la solicitud
        with open(os.path.join('data/video/', filename), 'wb') as handler:
            handler.write(vid.content)
            print(f"Archivo {filename} descargado exitosamente")
            NumDes = NumDes + 1
        time.sleep(1) #Tiempo de espera para evitar sobrecargas
    except requests.exceptions.Timeout:
        print(f"Error - Se agoto el tiempo de espera intentando descargar el archivo: {filename}.")
    except requests.exceptions.RequestException as e:
        print(f"Error mientras se descarga {filename}: {e}")    
    except Exception as e:
        print(f"Error inesperado descargando {filename}: {e}")

if NumDes == NumT:
    print("=== Todos los archivos de video de ThePocusAtlas se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")  

  0% (0 of 32) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo archivos de video de The POCUS Atlas...
Archivo 49_pocusatlas_covid.gif descargado exitosamente


  3% (1 of 32) |                         | Elapsed Time: 0:00:01 ETA:   0:00:53


Archivo 50_pocusatlas_covid.gif descargado exitosamente


  6% (2 of 32) |#                        | Elapsed Time: 0:00:14 ETA:   0:03:31


...

Archivo 79_pocusatlas_pneumonia.gif descargado exitosamente


 96% (31 of 32) |####################### | Elapsed Time: 0:00:58 ETA:   0:00:01


Archivo 80_pocusatlas_pneumonia.gif descargado exitosamente


100% (32 of 32) |########################| Elapsed Time: 0:01:00 Time:  0:01:00


=== Todos los archivos de video de ThePocusAtlas se descargaron correctamente ===
 32 archivos descargados


## 1.5. Radiopaedia

In [52]:
print('...Extrayendo archivos de vídeo de Radiopaedia...')
radio_df = metadata[metadata.source == 'Radiopaedia']
progress = ProgressBar(max_value=radio_df.shape[0]) # Barra de progreso
NumDes = 0
NumT = 0

for idx, row in progress(radio_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    try:
        # Escribiendo los archivos de vídeo en el disco
        vid = requests.get(row.url, timeout=10)   #.content
        vid.raise_for_status() #Verificación de errores en la solicitud
        with open(os.path.join('data/video/', filename), 'wb') as handler:
            handler.write(vid.content)
            print(f"Archivo {filename} descargado y guardado")
            NumDes = NumDes + 1
        time.sleep(1) #Tiempo de espera para evitar sobrecargas
    except requests.exceptions.Timeout:
        print(f"Error - Se agoto el tiempo de espera de la descarga de: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar {filename}: {e}")
    except Exception as e:
        print(f"Error inesperado descargando {filename}")

if NumDes == NumT:
    print("=== Todos los archivos de video de Radiopaedia se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")

  0% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo archivos de vídeo de Radiopaedia...
Archivo 152_radio_normal.mp4 descargado y guardado


 20% (1 of 5) |#####                     | Elapsed Time: 0:00:01 ETA:   0:00:05


Archivo 153_radio_pneumonia.mp4 descargado y guardado


 40% (2 of 5) |##########                | Elapsed Time: 0:00:03 ETA:   0:00:05


...

Archivo 155_radio_other.mp4 descargado y guardado


 80% (4 of 5) |####################      | Elapsed Time: 0:00:08 ETA:   0:00:02


Archivo 156_radio_other.mp4 descargado y guardado


100% (5 of 5) |##########################| Elapsed Time: 0:00:10 Time:  0:00:10


=== Todos los archivos de video de Radiopaedia se descargaron correctamente ===
 5 archivos descargados


## 1.6. CoreUltrasound

In [54]:
print('...Extrayendo archivos de vídeo de CoreUltrasound...')
core_df = metadata[metadata.source == 'CoreUltrasound']
progress = ProgressBar(max_value=core_df.shape[0]) # Barra de progreso
NumDes = 0
NumT = 0

for idx, row in progress(core_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    try:
        # Extraer video de Vimeo
        if 'vimeo' in row.url:
            v = Vimeo(row.url)
            print(f"Utilizando el enlace de descarga: {row.url}") #Imprimir el enlace de descarga
            stream = v.streams # Lista de transmisiones disponibles de diferente calidad
            highest_quality_available = stream[-1]
            print(f"Archivo {filename} descargado correctamente")
            highest_quality_available.download(download_directory = 'data/video/', filename = filename.split('.')[0])
            NumDes = NumDes + 1
        # Extraer videos mp4 de otros enlaces 
        else:
            # Escribir archivos de video en el disco 
            print(f"Utilizando el enlace de descarga: {row.url}") #Imprimir el enlace de descarga
            vid = requests.get(row.url, timeout=10) #.content
            vid.raise_for_status()
            with open(os.path.join('data/video/', filename), 'wb') as handler:
                handler.write(vid.content)
            print(f"Archivo {filename} descargado correctamente")
            NumDes = NumDes + 1
            time.sleep(3)
    except Exception as e:
        print(f"Ha ocurrido un error inesperado durante la descarga de {filename}: {e}")
     
if NumDes == NumT:
    print("=== Todos los archivos de video de CoreUltrasound se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")
 

  0% (0 of 18) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  5% (1 of 18) |#                        | Elapsed Time: 0:00:00 ETA:   0:00:01


...Extrayendo archivos de vídeo de CoreUltrasound...
Utilizando el enlace de descarga: https://vimeo.com/501985162
Ha ocurrido un error inesperado durante la descarga de 157_core_other.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
Utilizando el enlace de descarga: https://vimeo.com/472743963
Ha ocurrido un error inesperado durante la descarga de 158_core_pneumonia.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
Utilizando el enlace de descarga: https://vimeo.com/428684785


 16% (3 of 18) |####                     | Elapsed Time: 0:00:20 ETA:   0:01:40
 22% (4 of 18) |#####                    | Elapsed Time: 0:00:20 ETA:   0:01:10
 33% (6 of 18) |########                 | Elapsed Time: 0:00:20 ETA:   0:00:40


Ha ocurrido un error inesperado durante la descarga de 159_core_other.mp4: HTTPSConnectionPool(host='player.vimeo.com', port=443): Read timed out. (read timeout=None)
Utilizando el enlace de descarga: https://vimeo.com/428684785
Ha ocurrido un error inesperado durante la descarga de 160_core_other.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
Utilizando el enlace de descarga: https://vimeo.com/395474090
Ha ocurrido un error inesperado durante la descarga de 161_core_other.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
Utilizando el enlace de descarga: https://vimeo.com/395474090
Ha ocurrido un error inesperado durante la descarga de 162_core_other.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host r

 38% (7 of 18) |#########                | Elapsed Time: 0:00:23 ETA:   0:00:36


...

Utilizando el enlace de descarga: https://d25ixnv6uinqzi.cloudfront.net/post_files/uotw7.2.mp4
Archivo 172_core_other.mp4 descargado correctamente


 88% (16 of 18) |#####################   | Elapsed Time: 0:01:02 ETA:   0:00:07


Utilizando el enlace de descarga: https://d25ixnv6uinqzi.cloudfront.net/post_files/uotw6.1.mp4
Archivo 173_core_other.mp4 descargado correctamente


 94% (17 of 18) |######################  | Elapsed Time: 0:01:05 ETA:   0:00:03
100% (18 of 18) |########################| Elapsed Time: 0:01:05 Time:  0:01:05


Utilizando el enlace de descarga: https://vimeo.com/419341838
Ha ocurrido un error inesperado durante la descarga de 174_core_covid.mp4: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
=== No se descargaron todos los archivos de video ===
 11 videos descargados de 18


## 1.7. UF

In [21]:
#paper_df = metadata[(metadata.source == 'Paper') & ((metadata['id'].str.contains('199', na=False)) | (metadata['id'].str.contains('200', na=False)))] 

print("=== Descargando Archivos de video de UF ===")
NumDes = 0
NumT = 0

#Configuración de Selenium
download_dir = r'C:\Users\ldiaz\Desktop\4 - CON CAMBIOS COVID-US-main\data\video'
chrome_options = Options()
chrome_options.add_argument("--headless") #Modo sin interfaz
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59")
chrome_options.add_experimental_option("prefs", {"download.default_directory": download_dir,"download.prompt_for_download": False,"download.directory_upgrade": True,"safebrowsing.enabled": True})

#Ruta al driver de Chrome
service = Service("utils/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

#URLS a usar
urls = {"https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0001-VideoS1.mp4", "https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0002-VideoS2.avi"}

#Acceder al enlace para la descarga
for url in urls:
    NumT += 1
    filename = row.id + '.' + row.filetype
    driver.get(url) #Accede al enlace de descarga directamente
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(10) #Tiempo de espera para las cookies  
driver.quit()

#Revisar la correcta descarga del archivo
for url in urls:    
    filename = os.path.join(VIDEO_PATH, url.split("file=")[-1])
    if os.path.exists(filename):
        print(f"Descarga de {filename} terminada")
        NumDes += 1
    else:
        print(f"Ocurrio un error durante la descarga de {filename}")

if NumDes == NumT:
    print("=== Todos los archivos de video de la primera parte de UF se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT} ===")


=== Descargando Archivos de video de UF ===
Descarga de data/video/uog22034-sup-0001-VideoS1.mp4 terminada
Descarga de data/video/uog22034-sup-0002-VideoS2.avi terminada
=== Todos los archivos de video de la primera parte de UF se descargaron correctamente ===
 2 archivos descargados


In [ ]:
"""
#Diccionario de encabezados para el dominio utilizado
headers_dict = {'obgyn.onlinelibrary.wiley.com': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36','Referer': 'https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8', 'Accept-Language': 'en-US,en;q=0.9', 'Connection': 'keep-alive'}}


#URLs a utilizar
urls = {"https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0001-VideoS1.mp4", "https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0002-VideoS2.avi"}

session = requests.Session()
#Para saber si todas las descargas se realizaron correctamente  
Control = 0

for url in urls:
    filename = os.path.join(VIDEO_PATH, url.split("file=")[-1])
    domain = url.split("/")[2]
    
    #Asignar un Header dinamico
    headers = headers_dict.get(domain, {}).copy()
    headers['Referer'] = url #Cambia el referer a la URL completa
    
    try:
        
        #Ciclo para hacer 3 intentos en caso de errores
        for attempt in range(3):
            response = session.get(url, headers=headers, stream=True)
            print(f"Para el archivo {filename} ...")
            if response.status_code == 200:
                with open(filename, 'wb') as f:
                    shutil.copyfileobj(response.raw, f)
                print(f'Descarga de {filename} completada')
                Control = Control + 1
                break
            elif response.status_code == 403:
                print(f"Error 403 en intento {attempt + 1}. Reintentando...")
                time.sleep(random.randint(5, 10)) #Tiempo de espera aleatorio entre intentos
            else:
                print(f'Error al descargar {filename} - Estado : {response.status_code}')
                break
            #Delay aleatorio para evitar que el servidor bloquee solicitudes
            time.sleep(random.randint(3, 5))
    except requests.exceptions.RequestException as e:
        print(f"Error inesperado durante la descarga de {filename}: {e}")
if Control == len(urls):
    print("=== Archivos Descargados correctamente ===")
else:
    print(f"No se pudieron descargar todos los archivos - N° de archivos descargados: {Control}")
"""
    
#############################################################################################################   
    
"""paper_df = metadata[(metadata.source == 'Paper') & ((metadata['id'].str.contains('199', na=False)) | (metadata['id'].str.contains('200', na=False)))] 
progress = ProgressBar(max_value=paper_df.shape[0]) 

#Encabezados de los 2 dominios
headers_dict = {'obgyn.onlinelibrary.wiley.com':{'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 'refer':'https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0001-VideoS1.mp4'}}

for idx, row in progress(paper_df.iterrows()):
    
    filename = row.id + '.' + row.filetype
    print(f"Para el video correspondiente el enlace de descarga es: {row.url}")
    # Descarga de archivos 
    try:
        session = requests.Session()
        session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 'refer': ''})
        r = session.get(row.url, stream=True, timeout=10)
        #r = requests.get(row.url, stream=True, headers=headers, timeout=10)
        if r.status_code == 200:
            with open(os.path.join('data/video/', filename), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f) #Copia el contenido directamente al archivo local en modo binario 
            print(f"Archivo {filename} correctamente descargado")
        else:
            print(f"Error al descargar: Estado - {r.status_code}")
    except Exception as e:
        print(f"Error inesperado al descargar {filename}: {e}")
    
    # Estableciendo un delay aleatorio, de lo contrario la conexión se cerrara 
    delay = random.randint(3, 5)
    time.sleep(delay)
    
print('=== 2 archivos de video extra descargados! ===')        
"""

In [57]:
print('...Extrayendo archivos de video de UF...')
uf_df = metadata[metadata.source == 'UF']
NumDes = 0
NumT = 0

progress = ProgressBar(max_value=uf_df.shape[0]) 
#Descarga de archivos
for idx, row in progress(uf_df.iterrows()):
    filename = row.id + '.' + row.filetype 
    NumT = NumT + 1
    # Escribe los archivos de video en el disco
    try:
        r = requests.get(row.url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
        if r.status_code == 200:
            with open(os.path.join('data/video/', filename), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)  
                print(f"Archivo {filename} correctamente descargado")
                NumDes = NumDes + 1
        else:
            print(f"Error al descargar: Estado - {r.status_code}")     
    except requests.exceptions.RequestException as e:
        print(f"Se presento un error durante la descarga de {filename}: {e}")
        continue #Para continuar con los archivos restantes

    # Establece un delay aleatorio, de lo contrario se cerrara la conexión 
    delay = random.randint(3, 5)
    time.sleep(delay)

print(".......... Para la parte 2 de archivos de UF")
if NumDes == NumT:
    print("=== Todos los archivos de video de UF se descargaron correctamente ===")
    print(f" {NumT} archivos descargados")
elif NumDes > 0:
    print("=== No se descargaron todos los archivos de video ===")
    print(f" {NumDes} videos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo de video de {NumT}===")


   

  0% (0 of 24) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo archivos de video de UF...
Archivo 175_uf_normal.mp4 correctamente descargado


  4% (1 of 24) |#                        | Elapsed Time: 0:00:03 ETA:   0:01:25


Archivo 176_uf_normal.wmv correctamente descargado


  8% (2 of 24) |##                       | Elapsed Time: 0:00:08 ETA:   0:01:35


...

Archivo 197_uf_other.mp4 correctamente descargado


 95% (23 of 24) |####################### | Elapsed Time: 0:02:51 ETA:   0:00:07


Archivo 198_uf_other.wmv correctamente descargado


100% (24 of 24) |########################| Elapsed Time: 0:02:56 Time:  0:02:56


.......... Para la parte 2 de archivos de UF
=== Todos los archivos de video de UF se descargaron correctamente ===
 24 archivos descargados


## 1.8. Scientific Publications

In [14]:
print('...Extrayendo archivos de vídeo de "Scientific Publications"...')
paper_df = metadata[(metadata.source == 'Paper')] 

progress = ProgressBar(max_value=paper_df.shape[0]) 
NumDes = 0
NumT = 0

#Configuración de Selenium (en caso de ser necesario su uso)
download_dir = r'C:\Users\ldiaz\Desktop\4 - CON CAMBIOS COVID-US-main\data\video'
chrome_options = Options()
chrome_options.add_argument("--headless") #Modo sin interfaz
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59")
chrome_options.add_experimental_option("prefs", {"download.default_directory": download_dir,"download.prompt_for_download": False,"download.directory_upgrade": True,"safebrowsing.enabled": True})


#Inicializar el driver de Selenium
service = Service("utils/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

for idx, row in progress(paper_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    try:    
        #Hace la solicitud GET
        r = requests.get(row.url, stream=True, headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59'})
        print(f"Utilizando el enlace de descarga: {row.url}") #Imprimir el enlace de descarga
        time.sleep(5)
                
        #Verificar si se presento el error 403
        if r.status_code == 403:
            print("Accediendo al enlace mediante Selenium")
            driver.get(row.url)
            driver.execute_script("window.scrollBy(0, 300);") #Realiza desplazamientos en la pagina web para cargar contenido
            time.sleep(10)
            filepath = os.path.join(download_dir, filename)
            #Renombrar el archivo descargado
            with open(os.path.join('data/video/', filename), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
                NumDes += 1
            
            #Verificar que se haya descargado
            if os.path.exists(filepath):
                print(f"El archivo {filename} se descargo correctamente con Selenium")
            else:
                print(f"Error: el archivo {filename} no se pudo descargar")
        elif r.status_code == 200:
            #Guarda el archivo
            with open(os.path.join('data/video/', filename), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
            print(f"El archivo {filename} se descargo correctamente")
            NumDes = NumDes + 1
        
    except requests.exceptions.RequestException as e:
        print(f"Error inesperado durante la descarga de {filename}: {e}")
        continue #Para seguir con los demas archivos    
        
    # Establecer un delay aleatorio, de lo contrario se cierra la conexión
    if (('241_' in row.id) | ('242_' in row.id) | ('243_' in row.id)): #Delay mas grande para los ultimos archivos
        delay = random.randint(10, 20)
    else:
        delay = random.randint(3, 5)
    time.sleep(delay)

if NumDes == NumT:
    print('=== Archivos de video de Scientific Publications extraidos correctamente! ===')  
    print(f"=== {NumDes} archivos descargados ===")      
elif NumDes > 0:
    print("=== No se descargaron todos los archivos ===")
    print(f"{NumDes} archivos descargados de un total de {NumT}")
else:
    print("=== No se descargo ningun archivo ===")





...Extrayendo archivos de vídeo de "Scientific Publications"...


  0% (0 of 22) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


Utilizando el enlace de descarga: https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0001-VideoS1.mp4
Accediendo al enlace mediante Selenium
El archivo 199_paper_covid.mp4 se descargo correctamente con Selenium


  4% (1 of 22) |#                        | Elapsed Time: 0:00:22 ETA:   0:07:57


Utilizando el enlace de descarga: https://obgyn.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fuog.22034&file=uog22034-sup-0002-VideoS2.avi
Accediendo al enlace mediante Selenium
El archivo 200_paper_covid.avi se descargo correctamente con Selenium


...

Utilizando el enlace de descarga: https://static-content.springer.com/esm/art%3A10.1186%2Fcc5668/MediaObjects/13054_2007_5188_MOESM7_ESM.avi
El archivo 242_paper_other.avi se descargo correctamente


 95% (21 of 22) |######################  | Elapsed Time: 0:04:20 ETA:   0:00:12


Utilizando el enlace de descarga: https://static-content.springer.com/esm/art%3A10.1186%2Fcc5668/MediaObjects/13054_2007_5188_MOESM8_ESM.avi
El archivo 243_paper_other.avi se descargo correctamente


100% (22 of 22) |########################| Elapsed Time: 0:04:43 Time:  0:04:43


=== No se descargaron todos los archivos ===
21 archivos descargados de un total de 22


## 1.9. Clarius
* Extracción de la primera parte de los archivos clarius (**6 archivos**)

In [22]:
print('...Extrayendo archivos de video de Clarius...')
clarius_df = metadata[metadata.source == 'Clarius'].iloc[:6, :]

progress = ProgressBar(max_value=clarius_df.shape[0]) 
NumDes = 0 #Variable para numero de archivos descargados
NumT = 0 #Variable para numero total de archivos

for idx, row in progress(clarius_df.iterrows()):
    filename = row.id + '.' + row.filetype
    NumT = NumT + 1
    
    try:
        # Escribir los archivos de video en el disco
        r = requests.get(row.url, stream=True, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64)'})
        if r.status_code == 200:
            with open(os.path.join('data/video/', filename), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)       
            print(f"Archivo {filename} correctamente descargado")
            NumDes = NumDes + 1
        else:
            print(f"Error {r.status_code} mientras se intentaba descargar {filename}.")
    except requests.exceptions.RequestException as e:
        print(f"Error inesperado durante la descarga: {e}")
    
    # Establece un delay aleatorio, de lo contrario se cerrara la conexión
    delay = random.randint(3, 5)
    time.sleep(delay)
    
if NumDes == NumT:
    print('=== Archivos de video de Clarius extraidos correctamente! ===')        
    print(f"Total de archivos: {NumT}")
elif NumDes > 0:
    print("=== No todos los archivos se lograron descargar ===")
    print(f" {NumDes} archivos descargados de {NumT}")
else:
    print(f"=== No se descargo ningun archivo, de un total de: {NumT} ===")

    
    
    
    
    
    

  0% (0 of 6) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--


...Extrayendo archivos de video de Clarius...
Archivo 216_clarius_other.mp4 correctamente descargado


 16% (1 of 6) |####                      | Elapsed Time: 0:00:15 ETA:   0:01:18


Archivo 217_clarius_normal.mp4 correctamente descargado


 33% (2 of 6) |########                  | Elapsed Time: 0:00:22 ETA:   0:00:44


...

Archivo 220_clarius_covid.gif correctamente descargado


 83% (5 of 6) |#####################     | Elapsed Time: 0:00:39 ETA:   0:00:07


Archivo 221_clarius_covid.gif correctamente descargado


100% (6 of 6) |##########################| Elapsed Time: 0:00:45 Time:  0:00:45


=== Archivos de video de Clarius extraidos correctamente! ===
Total de archivos: 6


* Extracción de la segunda parte de los archivos Clarius (**17 Archivos**)

In [ ]:
import requests

print('Descargando archivos Zip de Clarius...')
file_id = '1bqsqNzAJYwdriOP9CcGWPzCUB-7G72Ta' # ID de enlace para compartir
zip_file_path = 'C:\\Users\\ldiaz\\Downloads\\clarius.zip' #os.path.join(get_download_path(), 'clarius.zip')
download_file_from_google_drive(file_id, zip_file_path)

# Extrae el archivo Zip
try:
    open_file = subprocess.Popen("utils/7z.exe" +' x ' + zip_file_path + ' -o' + 'data/video',stdout=subprocess.PIPE)
    result = open_file.communicate()
    if open_file.returncode != 0:
        print("Ha ocurrido un error durante la extracción del archivo Zip", result[0].decode())
    else:
        print('=== Archivos de video de Clarius extraidos correctamente! ===')
except Exception as e:
    print(f"Error durante la extracción: {e}")

#Eliminar Zip
try:
    os.remove(zip_file_path)
    print("El archivo Zip se elimino correctamente")
except Exception as e:
    print(f"Ha ocurrido un error al intentar eliminar el archivo Zip: {e}")

Descargando archivos Zip de Clarius...


/ |#                                                  | 0 Elapsed Time: 0:00:00
| |#                                                  | 0 Elapsed Time: 0:00:00


Descarga completa
Ha ocurrido un error durante la extracción del archivo Zip 
7-Zip 18.06 (x64) : Copyright (c) 1999-2018 Igor Pavlov : 2018-12-30

Scanning the drive for archives:
1 file, 2435 bytes (3 KiB)

Extracting archive: C:\Users\ldiaz\Downloads\clarius.zip

Can't open as archive: 1
Files: 0
Size:       0
Compressed: 0



# 2. Procesamiento de Videos

#### Mover los archivos de video originales a la carpeta original

In [7]:
source_dir = 'data/video/'
target_dir = 'data/video/original' #Direccion nueva para guardar los archivos

file_names = os.listdir(source_dir)

#Opcion para crear la carpeta en caso de que no existiera
if not os.path.exists('data/video/original'): 
    os.makedirs('data/video/original') 

progress = ProgressBar() #Barra de progreso
for file_name in progress(file_names):
    if file_name.endswith(('.mp4', '.avi', '.gif', '.wmv', '.mov')): #Tipos de video soportados
        shutil.move(os.path.join(source_dir, file_name), target_dir)

  0% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


## 2.1. Obtención de propiedades de los archivos de video

In [8]:
VIDEO_PATH_ORG = 'data/video/original/' #Carpeta con la que se va a trabajar

vid_files = os.listdir(VIDEO_PATH_ORG)

progress = ProgressBar(max_value=metadata.shape[0]) 
with open('utils/video_files_properties.csv', 'w') as f:
    # Escribe el encabezado del archivo
    f.write('filename,framerate,width,height,frame_count,duration_secs\n')
    
    # Recorrer los archivos de vídeo y obtener sus propiedades 
    for vid in progress(vid_files):
        try:
            vid_filename = VIDEO_PATH_ORG + str(vid)
            file_type = vid.split('.')[-1]

            # Obtener las propiedades de archivos de video
            cv2video = cv2.VideoCapture(vid_filename)
            #En caso de no encontrar el archivo
            if not cv2video.isOpened():
                raise IOError(f"No se ha podido abrir el archivo: {vid}") #Genera una excepcion cuando no se abre el archivo
            
            height = cv2video.get(cv2.CAP_PROP_FRAME_HEIGHT)
            width  = cv2video.get(cv2.CAP_PROP_FRAME_WIDTH) 
            frame_rate = round(cv2video.get(cv2.CAP_PROP_FPS), 2)

            #Extraer propiedades (Numero total de cuadros y Duración) para formatos mp4 y gif
            if file_type == 'mp4':
                frame_count = cv2video.get(cv2.CAP_PROP_FRAME_COUNT) 
                duration = round((frame_count / frame_rate), 2)
            elif file_type == 'gif':
                frame_count = round(Image.open(vid_filename).n_frames) #round((duration * frame_rate ), 0)
                duration = round((frame_count / frame_rate), 2)

            # Escribe las propiedades del vídeo en el archivo
            line_to_write = str(vid) + ',' + str(frame_rate) + ',' + str(width) + ',' + str(height) + ',' + str(frame_count) + ',' + str(duration) + '\n'
            f.write(line_to_write)            
        except Exception as e:
            print(f"Error al procesar {vid}: {e}")

print(f"Se extrajeron las propiedades de los archivos correctamente")



  0% (0 of 244) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


  0% (1 of 244) |                        | Elapsed Time: 0:00:00 ETA:   0:01:08
  1% (4 of 244) |                        | Elapsed Time: 0:00:00 ETA:   0:00:20
  4% (10 of 244) |                       | Elapsed Time: 0:00:00 ETA:   0:00:10
  5% (13 of 244) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:09
  7% (19 of 244) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:07
  9% (22 of 244) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:07
 10% (25 of 244) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:06
 11% (28 of 244) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:06
 12% (31 of 244) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:05
 13% (34 of 244) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:05
 15% (38 of 244) |###                    | Elapsed Time: 0:00:00 ETA:   0:00:05
 18% (44 of 244) |####                   | Elapsed Time: 0:00:01 ETA:   0:00:04
 19% (47 of 244) |####                  

Se extrajeron las propiedades de los archivos correctamente


## 2.2. Preprocesamiento de Video

In [8]:
#Creación de la carpeta "Cropped" para guardar los archivos de video que se recorten 
VIDEO_CROPPED_OUT = 'data/video/cropped/' #processed/cropped/'

# Crea la carpeta si aun no existe 
if not os.path.exists('data/video/cropped'): #processed/cropped'):
    os.makedirs('data/video/cropped') #processed/cropped')

### 2.2.1. Recorte Inicial

In [9]:
#Carga el archivo CSV con metadatos de recorte de videos
# Leer el archivo de metadatos de recorte 
vid_crp_metadata = pd.read_csv('utils/video_cropping_metadata.csv', sep=',', encoding='latin1')
print(vid_crp_metadata.shape)
vid_crp_metadata.head(2)

(243, 27)


,filename,source,probe,class,org_width,org_height,org_framecount,org_framerate,org_duration,green_dot,...,del_upper,width_rate,x1_w_y1_h,cropped_filename,crp_width,crp_height,version,date_added,multiple_videos,Note
0,1_butterfly_covid.mp4,Butterfly,Convex,COVID,880,1080,65,19.57,3.32,no,...,15.0,0.035,NaN,1_butterfly_covid_prc.avi,820.0,820.0,1.0,Nov_2020,NaN,NaN
1,2_butterfly_covid.mp4,Butterfly,Convex,COVID,720,1236,818,30.00,27.27,yes,...,83.0,0.068,NaN,2_butterfly_covid_prc.avi,624.0,624.0,1.0,Nov_2020,NaN,NaN


In [12]:
progress = ProgressBar(max_value=vid_crp_metadata.shape[0])

for idx, row in progress(vid_crp_metadata.iterrows()):
    
    try:
        vid_arr = []  # Arreglo para almacenar fotogramas de archivos de video

        filename = row.filename
        file_label = filename.split('_')[-1].split('.')[0] # Etiqueta de archivos de video

        # El siguiente archivo fue eliminado de en la nueva versión de datos Butterfly
        if filename == '22_butterfly_covid.mp4':
            continue #Se salta al siguiente archivo
    
        try:
            cap = cv2.VideoCapture(os.path.join(VIDEO_PATH_ORG, filename))
            if not cap.isOpened():
                print(f"No se pudo abrir el archivo: {filename}")
                continue #Se salta al siguiente archivo
        except Exception as e:
            print(f"Error al intentar abrir archivo: {filename}: {e}")
            continue #Pasa al siguiente archivo
        
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
        dim = (width, height) # Dimension del archivo original 
    
        if pd.isna(row.x1_w_y1_h): # Recorte cuadrado (Square cropping)
            
            try:
                DEL_UPPER = int(row.del_upper) # Remover la parte superior
                WIDTH_RATE = float(row.width_rate) # Para quitar lados, por ejemplo el medidor 

                width_border = int(width * WIDTH_RATE)
                width_box = int(width - (2 * width_border)) 
                if width_box + DEL_UPPER > height:
                    width_box = int(height - DEL_UPPER)
                    width_border = int( (width / 2) - (width_box / 2))

                while(True):
                    ret, frame = cap.read()

                    if not ret:
                        break

                    # Recortar
                    frame = frame[DEL_UPPER:width_box + DEL_UPPER, width_border:width_box + width_border]
                    #Convierte fotogramas en arreglos haciendolo compatible con cv2
                    frame = np.asarray(frame).astype(np.uint8)
                    vid_arr.append(frame)
            except Exception as e:
                print(f"Ocurrio un error al intentar recortar {filename} con recorte cuadrado: {e}")    
                continue
                
        else: # Recortar utilizando coordenadas (x1,y1) y (x2, y2). La salida no sera necesariamente un archivo cuadrado
            try:
                X1 = int(row.x1_w_y1_h.split(',')[0].replace('(', ''))
                W = int(row.x1_w_y1_h.split(',')[1].strip())
                Y1 = int(row.x1_w_y1_h.split(',')[2].strip())
                H = int(row.x1_w_y1_h.split(',')[3].replace(')', '').strip())

                while(True):
                    ret, frame = cap.read()

                    if not ret:
                        break

                    # Recortar
                    frame = frame[Y1:Y1 + H, X1:X1 + W]

                    frame = np.asarray(frame).astype(np.uint8)
                    vid_arr.append(frame)
            except Exception as e:
                print(f"Ocurrio un erro al intentar recortar {filename} mediante coordenadas: {e}")
                continue

        #Almacenar video recortado
        try:
            vid_arr = np.asarray(vid_arr)
            prc_dim = vid_arr.shape[1:3] # Dimension del archivo recortado
            prc_dim = (prc_dim[1], prc_dim[0])

            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(os.path.join(VIDEO_CROPPED_OUT + filename.split('.')[0] + '_prc.avi'), fourcc, 20.0, tuple(prc_dim))

            for frame in vid_arr:
                out.write(frame.astype("uint8"))

            vid_crp_metadata.iloc[idx, vid_crp_metadata.columns.get_loc('crp_width')] = prc_dim[1]
            vid_crp_metadata.iloc[idx, vid_crp_metadata.columns.get_loc('crp_height')] = prc_dim[0]

            cap.release()
            out.release()
            cv2.destroyAllWindows()
        except Exception as e:
            print(f"Ocurrio un error al intentar guardar el video recortado: {e}")
            
            
    except Exception as e:
        print(f"Error durante el procesamiento del archivo: {filename}: {e}")

try:
    vid_crp_metadata.to_csv('utils/video_cropping_metadata.csv', index=None)
    print('Recortes iniciales terminados...')
except Exception as e:
    print(f"Error al guardar el archivo de metadatos: {e}")

  0% (0 of 243) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 243) |                        | Elapsed Time: 0:00:01 ETA:   0:06:58
  0% (2 of 243) |                        | Elapsed Time: 0:00:12 ETA:   0:25:14
  1% (3 of 243) |                        | Elapsed Time: 0:00:14 ETA:   0:19:59
  1% (4 of 243) |                        | Elapsed Time: 0:00:16 ETA:   0:16:34
  2% (5 of 243) |                        | Elapsed Time: 0:00:19 ETA:   0:15:45
  2% (6 of 243) |                        | Elapsed Time: 0:00:25 ETA:   0:17:02
  2% (7 of 243) |                        | Elapsed Time: 0:00:27 ETA:   0:15:34
  3% (8 of 243) |                        | Elapsed Time: 0:00:29 ETA:   0:14:21
  3% (9 of 243) |                        | Elapsed Time: 0:00:31 ETA:   0:13:33
  4% (10 of 243) |                       | Elapsed Time: 0:00:32 ETA:   0:12:40
  4% (11 of 243) |#                      | Elapsed Time: 0:00:33 ETA:   0:11:44
  4% (12 of 243) |#                     

No se pudo abrir el archivo: 161_core_other.mp4
No se pudo abrir el archivo: 162_core_other.mp4


 67% (163 of 243) |##############        | Elapsed Time: 0:02:12 ETA:   0:01:04
 67% (164 of 243) |##############        | Elapsed Time: 0:02:12 ETA:   0:01:03
 67% (165 of 243) |##############        | Elapsed Time: 0:02:13 ETA:   0:01:03
 68% (166 of 243) |###############       | Elapsed Time: 0:02:14 ETA:   0:01:02
 68% (167 of 243) |###############       | Elapsed Time: 0:02:14 ETA:   0:01:01
 69% (168 of 243) |###############       | Elapsed Time: 0:02:15 ETA:   0:01:00
 69% (169 of 243) |###############       | Elapsed Time: 0:02:16 ETA:   0:00:59
 69% (170 of 243) |###############       | Elapsed Time: 0:02:17 ETA:   0:00:58
 70% (171 of 243) |###############       | Elapsed Time: 0:02:17 ETA:   0:00:57
 70% (172 of 243) |###############       | Elapsed Time: 0:02:18 ETA:   0:00:57
 71% (173 of 243) |###############       | Elapsed Time: 0:02:19 ETA:   0:00:56
 71% (174 of 243) |###############       | Elapsed Time: 0:02:19 ETA:   0:00:55
 72% (175 of 243) |###############      

Recortes iniciales terminados...


# 3. Extracción de imagenes de ultrasonidos a partir de vídeos

#### Leer las propiedades de video

In [10]:
#Carga el archivo CSV en un DataFrame
vid_prop_df = pd.read_csv('utils/video_files_properties.csv')

# Fusionar con el archivo de video de metadatos
#Convierte la columna con los nombres a tipo String y elimina espacios
vid_prop_df['filename'] = vid_prop_df['filename'].astype(str).str.strip()

#Crea una columna temporal en el DataFrame combinando ID y FileType
metadata['filename2'] = metadata['id'].str.strip()+ '.' + metadata['filetype'].str.strip()

#Fusiona 'Vid_prop_df' y 'metadata'
vid_prop_df = pd.merge(vid_prop_df, metadata[['filename2', 'source', 'probe', 'class']], left_on='filename', right_on='filename2', how='left').drop('filename2', axis=1)

del metadata['filename2'] #Elimina la columna temporal 'filename2'
print(vid_prop_df.shape) #Imprime (Filas y columnas) de 'Vid_prop_df' despues de la fusión
vid_prop_df.head(2) #Muestra las 2 primeras filas resultantes

(240, 9)


,filename,framerate,width,height,frame_count,duration_secs,source,probe,class
0,100_litfl_other.mp4,15.0,480.0,360.0,46.0,3.07,Litfl,Convex,Other
1,101_litfl_other.mp4,15.0,480.0,360.0,28.0,1.87,Litfl,Convex,Other


#### Extracción de fotogramas de vídeos originales
* v1.4.: 32,052 images are extracted
* v1.3.: 19,161 images are extracted
* v1.2.: 15,282 images are extracted

In [11]:
#Define la ruta como directorio para guardar las imagenes obtenidas de los videos originales
IMAGE_PATH_ORG = 'data/image/original/'

# En caso de no existir, crea la carpeta para las imagenes extraídas de los videos originales
if not os.path.exists(IMAGE_PATH_ORG):
    os.makedirs(IMAGE_PATH_ORG)

In [18]:
import importlib
import image_data
importlib.reload(image_data)
from image_data import extract_images # Modulo personalizado - contiene la función de extracción de imágenes

extract_images(video_path= VIDEO_PATH_ORG, image_path=IMAGE_PATH_ORG, cropped=False)

  0% (0 of 240) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Realizando extracción de imagenes


  0% (1 of 240) |                        | Elapsed Time: 0:00:00 ETA:   0:01:41
  0% (2 of 240) |                        | Elapsed Time: 0:00:00 ETA:   0:01:16
  1% (3 of 240) |                        | Elapsed Time: 0:00:00 ETA:   0:01:09
  1% (4 of 240) |                        | Elapsed Time: 0:00:01 ETA:   0:01:43
  2% (5 of 240) |                        | Elapsed Time: 0:00:02 ETA:   0:01:59
  2% (6 of 240) |                        | Elapsed Time: 0:00:02 ETA:   0:01:56
  2% (7 of 240) |                        | Elapsed Time: 0:00:03 ETA:   0:01:55
  3% (8 of 240) |                        | Elapsed Time: 0:00:04 ETA:   0:02:17
  3% (9 of 240) |                        | Elapsed Time: 0:00:05 ETA:   0:02:13
  4% (10 of 240) |                       | Elapsed Time: 0:00:05 ETA:   0:02:06
  4% (11 of 240) |#                      | Elapsed Time: 0:00:12 ETA:   0:04:15
  5% (12 of 240) |#                      | Elapsed Time: 0:00:12 ETA:   0:04:01
  5% (13 of 240) |#                     

## 3.1. Extracción de fotogramas de archivos de video recortados

In [12]:
IMAGE_CROPPED_OUT = 'data/image/cropped/'
IMAGE_MASK_OUT = 'data/mask/'

# Crea carpetas de imágenes recortadas y pintadas, ademas de la carpeta de mascara (mask) si aun no existe
if not os.path.exists(IMAGE_CROPPED_OUT):
    os.makedirs(IMAGE_CROPPED_OUT)
if not os.path.exists(IMAGE_MASK_OUT):
    os.makedirs(IMAGE_MASK_OUT)

In [20]:
import importlib
import image_data
importlib.reload(image_data)
from image_data import extract_images # Modulo personalizado - contiene la función de extracción de imágenes
extract_images(video_path= VIDEO_CROPPED_OUT, image_path=IMAGE_CROPPED_OUT, cropped=True)

  0% (0 of 243) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Realizando extracción de imagenes


  0% (1 of 243) |                        | Elapsed Time: 0:00:02 ETA:   0:08:30
  0% (2 of 243) |                        | Elapsed Time: 0:00:20 ETA:   0:40:20
  1% (3 of 243) |                        | Elapsed Time: 0:00:24 ETA:   0:32:56
  1% (4 of 243) |                        | Elapsed Time: 0:00:28 ETA:   0:27:56
  2% (5 of 243) |                        | Elapsed Time: 0:00:35 ETA:   0:28:14
  2% (6 of 243) |                        | Elapsed Time: 0:00:49 ETA:   0:32:28
  2% (7 of 243) |                        | Elapsed Time: 0:00:53 ETA:   0:29:55
  3% (8 of 243) |                        | Elapsed Time: 0:00:56 ETA:   0:27:46
  3% (9 of 243) |                        | Elapsed Time: 0:01:00 ETA:   0:26:14
  4% (10 of 243) |                       | Elapsed Time: 0:01:03 ETA:   0:24:40
  4% (11 of 243) |#                      | Elapsed Time: 0:01:05 ETA:   0:23:03
  4% (12 of 243) |#                      | Elapsed Time: 0:01:12 ETA:   0:23:21
  5% (13 of 243) |#                     

Omitiendo el archivo recortado 160_core_other_prc.avi


 67% (163 of 243) |##############        | Elapsed Time: 0:04:53 ETA:   0:02:23
 67% (164 of 243) |##############        | Elapsed Time: 0:04:54 ETA:   0:02:21
 67% (165 of 243) |##############        | Elapsed Time: 0:04:56 ETA:   0:02:20
 68% (166 of 243) |###############       | Elapsed Time: 0:04:58 ETA:   0:02:18
 68% (167 of 243) |###############       | Elapsed Time: 0:04:59 ETA:   0:02:16
 69% (168 of 243) |###############       | Elapsed Time: 0:05:01 ETA:   0:02:14
 69% (169 of 243) |###############       | Elapsed Time: 0:05:02 ETA:   0:02:12
 69% (170 of 243) |###############       | Elapsed Time: 0:05:04 ETA:   0:02:10
 70% (171 of 243) |###############       | Elapsed Time: 0:05:06 ETA:   0:02:08
 70% (172 of 243) |###############       | Elapsed Time: 0:05:08 ETA:   0:02:07
 71% (173 of 243) |###############       | Elapsed Time: 0:05:09 ETA:   0:02:05
 71% (174 of 243) |###############       | Elapsed Time: 0:05:11 ETA:   0:02:03
 72% (175 of 243) |###############      

### 3.1.1. (Opcional) Extracción de fotogramas a partir de videos de ultrasonido a partir de parámetros como filtro 
* Puede extraer imágenes a partir de los siguientes parámetros
    * Número máximo de fotogramas que se extraerán de cada archivo de vídeo
    * Extracción de un conjunto especifico de clases ['COVID', 'Neumonia', 'Normal', 'Otro']
    * Extracción de un conjunto especifico de fuentes de datos ['Butterfly', 'GrepMed', 'LITFL', 'PocusAtlas', 'CU', 'Radiopaedia', 'UF', 'Paper', 'Clarius']
    * Extracción de un conjunto especifico de sondas ['convexa', 'lineal']

In [ ]:
#extract_images(video_path= VIDEO_CROPPED_OUT, image_path=IMAGE_CROPPED_OUT, cropped=True, 
#                max_frames=10, 
#                target_class=['COVID', 'Pneumonia', 'Normal'],
#                target_source=['Butterfly', 'GrepMed', 'LITFL', 'PocusAtlas'],
#                target_probe=['convex', 'linear']))

## 3.2. Preprocesamiento de Imagenes

#### Lectura de metadatos de preprocesamiento de imágenes 

In [13]:
image_prc_df = pd.read_csv('utils/mask_metadata.csv')

image_prc_df = image_prc_df[image_prc_df.filename !='22_butterfly_covid.mp4'] # 22_butterfly_covid.mp4 fue eliminado de Butterfly en marzo 

print(image_prc_df.shape)
image_prc_df.head(2)

(242, 17)


,filename,source,probe,class,org_width,org_height,cropped_filename,crp_width,crp_height,need_mask_after_crop,need_multiple_masks,frame_specific_masks,delete_frames_from_to,mask_main_filename,tight_inpainting,version,date_added
0,1_butterfly_covid.mp4,Butterfly,Convex,COVID,880,1080,1_butterfly_covid_prc.avi,820,820,yes,no,NaN,NaN,1_butterfly_covid_prc_convex_frame0_mask.jpg,no,1.0,Nov_2020
1,2_butterfly_covid.mp4,Butterfly,Convex,COVID,720,1236,2_butterfly_covid_prc.avi,624,624,yes,yes,"118-130, 134-139, 147-150","131-133, 143-146, 154-202, 210-813",2_butterfly_covid_prc_convex_frame0_mask.jpg,no,1.0,Nov_2020


### 3.2.1. Eliminación de fotogramas con artefactos en el ROI
Es necesaria la eliminación de algunos fotogramas en los siguientes archivos, debido a que el puntero en movimiento esta en ROI. Se eliminara de las carpetas de imágenes recortadas.
* 2_butterfly_covid.mp4
* 6_butterfly_covid.mp4
* 16_butterfly_covid.mp4
* 20_butterfly_normal.mp4
* 22_butterfly_covid.mp4 (Se elimino en la publicación de marzo los datos de butterfly)
* 25_grepmed_pneumonia.mp4
* 178_uf_other.mp4 (Se eliminaron los 30 fotogramas iniciales)
* 184_uf_pneumonia.mp4 (Se eliminaron los 30 fotogramas iniciales)

Se necesitan 2 mascaras para los siguientes videos
* 178_uf_other.mp4 
* 184_uf_pneumonia.mp4

**Número de fotogramas:**
* __Número total inicial de fotogramas:__ 
    * v1.4.: 32,052
    * v1.3.: 19,161
    * v1.2.: 13,646
* __Número total de fotogramas despues de la eliminación:__ 
    * v1.4.: 29,651
    * v1.3.: 16,822
    * v1.2.: 11,307

In [15]:
progress = ProgressBar(max_value=image_prc_df[~pd.isna(image_prc_df.delete_frames_from_to)].shape[0])

for idx, row in progress(image_prc_df[~pd.isna(image_prc_df.delete_frames_from_to)].iterrows()):
    frames_to_delete = row.delete_frames_from_to.strip().split(',')
    frame_name_main = row.mask_main_filename.split('.')[0].split('_frame')[0]
    
    for frames in frames_to_delete:
        from_frame = int(frames.split('-')[0])
        to_frame = int(frames.split('-')[1]) + 1
        
        # Eliminar fotogramas con partes móviles en el ROI 
        for i in range(from_frame, to_frame):
            file_to_remove = IMAGE_CROPPED_OUT + frame_name_main + '_frame' + str(i) + '.jpg'
            #Verificar si el archivo existe
            if os.path.exists(file_to_remove):
                try:
                    os.remove(file_to_remove)
                except Exception as e:
                    print(f"Ocurrio un erro intentando eliminar {file_to_remove}: {e}")
            else:
                print(f"El archivo {file_to_remove} no existe, por lo tanto no se pudo eliminar")
                
print("=== Archivos con artefactos en el ROI eliminados correctamente! ===")

  0% (0 of 7) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--


 14% (1 of 7) |###                       | Elapsed Time: 0:00:00 ETA:   0:00:00
 28% (2 of 7) |#######                   | Elapsed Time: 0:00:00 ETA:   0:00:00


El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame131.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame132.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame133.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame143.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame144.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame145.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame146.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame154.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/2_butterfly_covid_prc_convex_frame155.jpg 

 42% (3 of 7) |###########               | Elapsed Time: 0:00:00 ETA:   0:00:00
 57% (4 of 7) |##############            | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (7 of 7) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame610.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame611.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame612.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame613.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame614.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame615.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame616.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_frame617.jpg no existe, por lo tanto no se pudo eliminar
El archivo data/image/cropped/16_butterfly_covid_prc_convex_fram

### 3.2.2. Aplicación de mascaras

In [14]:
CLEAN_IMAGE_OUT = 'data/image/clean/'
CLEAN_VIDEO_OUT = 'data/video/clean/'

# Si no existen, crea carpetas de imágenes y vídeos limpios (Clean)
if not os.path.exists(CLEAN_IMAGE_OUT):
    os.makedirs(CLEAN_IMAGE_OUT)
if not os.path.exists(CLEAN_VIDEO_OUT):
    os.makedirs(CLEAN_VIDEO_OUT)

In [ ]:
def zero_pad_array(arr, pad=5):
    '''
    Agrega un marco de ceros alrededor de la matriz de entrada
    
    Parametros -> arr - matriz de entrada
               -> pad - Cantidad de pixeles de relleno en cada lado 
    
    Al final regresa la matriz con relleno aplicado 
    '''
    if len(arr.shape) == 3: #Imagen con canales de color
        padded_arr = np.zeros((arr.shape[0]+2*pad, arr.shape[1]+2*pad, arr.shape[2]), dtype=np.uint8)
        padded_arr[pad:pad + arr.shape[0], pad:pad + arr.shape[1], :] = arr
    else: #Matriz 2D - (Escala de grises)
        padded_arr = np.zeros((arr.shape[0]+2*pad, arr.shape[1]+2*pad), dtype=np.uint8)
        padded_arr[pad:pad + arr.shape[0], pad:pad + arr.shape[1]] = arr
    return padded_arr
        

def frame_inpainting(frame_dict, mask, default_mask=0, kernel_size=(5,5), method='telea', pad=5):
    '''
    Realiza la restauración de los fotogramas mediante la creación de mascaras, para eliminar artefactos o reconstruir areas dañadas
    
    Parametros:
        -> frame_dict: diccionario de fotogramas del vídeo, registrados segun el numero de fotograma
    
        -> mask: mascara para identificar areas a restaurar, arreglo (h, w, 1) si hay una sola mascara, 
            de lo contrario un diccionario de las matrices registradas por numero de fotograma 
    
        -> default_mask: Indice de la mascara predeterminada, para fotogramas sin máscara específica
    
        -> kernel_size: tamaño del parche utilizado para dilatar la mascara y realizar la restauración
    
        -> method: Metodo de restauración 'ns' (navier-stokes) o 'telea' 
        
        ->pad: Cantidad de pixeles para relleno en los bordes 
        
    Al final regresa un diccionario con los fotogramas restaurados
    '''
    # Dilata la mascara y se asegura de que cubra lo suficiente el ROI para que quede enmascarada
    #Crea el kernel 
    kernel = np.ones(kernel_size, np.uint8) 
    
    #Convierte la mascara en un diccionario en caso de que no lo sea
    if type(mask) is not dict: 
        mask = {default_mask: mask}
    
    #Dilatar y rellenar las máscaras
    #masks_processed = {key:cv2.dilate(zero_pad_array(m, pad=pad), kernel, iterations=1) for key, m in mask.items()}
    masks_processed = {}
    for key, m in mask.items():
        padded_mask = zero_pad_array(m, pad=pad)
        #Redimensionar de ser necesario
        frame_shape = next(iter(frame_dict.values())).shape[:2]
        if padded_mask.shape[:2] != frame_shape:
            #print(f"Redimensionando la máscara {key} para que coincida con el fotograma.")
            padded_mask = cv2.resize(padded_mask, (frame_shape[1], frame_shape[0]), interpolation=cv2.INTER_NEAREST)
        masks_processed[key] = cv2.dilate(padded_mask, kernel, iterations=1)
    
    
    #Mapeo de métodos de restauración disponibles
    method_dict = {'ns':cv2.INPAINT_NS, 'telea':cv2.INPAINT_TELEA}
    
    #Diccionario para almacenar los fotogramas restaurados
    frames_inpainted = {}
    
    for key, frame in frame_dict.items():
        padded_frame = zero_pad_array(frame, pad=pad)
        if key in masks_processed: #Restaurar usando la mascara correspondiente
            #print(frame.shape, masks_processed[key].shape)
            #frames_inpainted[key] = cv2.inpaint(zero_pad_array(frame, pad=pad), masks_processed[key], 3, method_dict[method])[pad:-pad, pad:-pad, :]
            mask_to_use = masks_processed[key]
            mask_to_use = cv2.resize(mask_to_use, (padded_frame.shape[1], padded_frame.shape[0]), interpolation=cv2.INTER_NEAREST)
        else: # Mascara predeterminada
            #frames_inpainted[key] = cv2.inpaint(zero_pad_array(frame, pad=pad), masks_processed[default_mask], 3, method_dict[method])[pad:-pad, pad:-pad, :]
            mask_to_use = masks_processed[default_mask]
            mask_to_use = cv2.resize(mask_to_use, (padded_frame.shape[1], padded_frame.shape[0]), interpolation=cv2.INTER_NEAREST)
            
        #print(f"Las dimensiones son: tamaño del fotograma: {padded_frame.shape}, tamaño de la mascara utilizada: {mask_to_use.shape}")
        #Restauracion del fotograma
        frames_inpainted[key] = cv2.inpaint(padded_frame, mask_to_use, 3, method_dict[method])[pad:-pad, pad:-pad, :]
    return frames_inpainted




In [17]:
progress = ProgressBar(max_value=image_prc_df.shape[0])

#Función para generar videos limpios
def generate_clean_video(frames, output_path):
    """
    Genera un video limpio a partir de los fotogramas
    
    Argumentos:
        -> Frames (dict): Diccionario de fotogramas ordenados por numero de fotograma.
        -> Output_path (str): Ruta de salida para el video generado
    
    """
    keys = sorted(frames.keys()) #Asegura el orden de los fotogramas
    clean_vid_frames = [frames[k] for k in keys]
    
    #Obtener el tamaño del video basandose en el primer fotograma
    h, w, layers = clean_vid_frames[0].shape
    size = (w, h)
    
    #Escribe los fotogramas en el archivo de video...
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    for frame in clean_vid_frames:
        out.write(frame)
    out.release()
    

#Realiza la iteracion en cada fila del Dataframe con los metadatos del procesamiento
for idx, row in progress(image_prc_df.iterrows()):     
    
    #Omitir el video 160
    if row.filename == '160_core_other.mp4' or row.filename == '161_core_other.mp4' or row.filename == '162_core_other.mp4':
        print("Archivo no encontrado.... Continuando el proceso")
        continue #Salto al siguiente video
    
    # Obtener el token principal del nombre del archivo - basandose en el tipo de sonda
    if row.probe == 'Convex':
        filename_main = row.filename.split('.')[0] + '_prc_convex'
    elif row.probe == 'Linear':
        filename_main = row.filename.split('.')[0] + '_prc_linear'
    
    
    #cropped_files = [f for f in os.listdir(IMAGE_CROPPED_OUT) if f.startswith(filename_main)]
    #if not cropped_files:
    #    print(f"Advertencia- No se encontraron máscaras para {row.filename}. Omitiendo el archivo.....")
    #    continue
    
    '''
    #Define el tamaño del parche de repintado - a partir de la proximidad de objetos al ROI
    if row.tight_inpainting == 'yes':
        # Objetos cercanos al ROI, evita el desbordamiento durante el repintado
        inpainting_kernel_size = (1,1)
    else:
        # No hay objetos cerca del ROI, repintado mas efectivo
        inpainting_kernel_size = (5,5)
    '''
    inpainting_kernel_size = (1, 1) if row.tight_inpainting == 'yes' else (5, 5)



    # Verificar si los fotogramas recortados no necesitan limpieza 
    if row.need_mask_after_crop == 'no':    
        frames = {}
        # Caso 1. Los fotogramas no necesitan limpieza
        # Copia las imagenes recortadas y renombrandolas en la carpeta limpia 
        for file in os.listdir(IMAGE_CROPPED_OUT):
            if file.startswith(filename_main):
                #last_part = file.split('_')[-1]
                #last_part = last_part.replace('frame', '_clean_frame')
                new_filename = file.replace('frame', 'clean_frame') #Renombrar el archivo
                #print(file, new_filename) #, last_part)
                shutil.copy(IMAGE_CROPPED_OUT + file, CLEAN_IMAGE_OUT + new_filename)

                #Carga cada fotograma en un diccionario para crear el video
                img = cv2.imread(os.path.join(CLEAN_IMAGE_OUT, new_filename))
                frame_num = int(re.search(r'\d+$', file[:-4]).group())
                frames[frame_num] = img
        
        #Genera y escribe el video limpio a partir de los fotogramas
        generate_clean_video(frames, os.path.join(CLEAN_VIDEO_OUT, filename_main + '_clean.avi'))
        
        '''
        # Crea un video limpio a partir de los fotogramas
        keys = list(frames.keys())S
        keys.sort() #Asegura el orden correcto de los fotogramas
        clean_vid_frames = [frames[k] for k in keys]

        #Configura los parametros del video
        h, w, layers = clean_vid_frames[0].shape
        size = (w, h)

        #Escribe los fotogramas en un archivo de video limpio
        out = cv2.VideoWriter(os.path.join(CLEAN_VIDEO_OUT + filename_main + '_clean.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
        for i in range(len(clean_vid_frames)):
            out.write(clean_vid_frames[i])
        out.release()    
        '''
        
    # Caso 2. Fotogramas que necesitan limpieza
    else: 
        # Crea un diccionario de fotogramas
        frames = {}
        for f in os.listdir(IMAGE_CROPPED_OUT):
            if f.startswith(filename_main):
                img = cv2.imread(os.path.join(IMAGE_CROPPED_OUT, f))
                frame_num = int(re.search(r'\d+$', f[:-4]).group())
                frames[frame_num] = img

        # Comprueba si el archivo de video requiere multiples mascaras
        if row.need_multiple_masks == 'no':
            #Carga una unica mascara principal
            mask = cv2.imread(os.path.join(IMAGE_MASK_OUT, filename_main + '_frame0_mask.jpg'))[:,:,0]

            # Realiza retoques en los fotogramas utilizando la mascara principal
            frames_inpainted = frame_inpainting(frames, mask, kernel_size=inpainting_kernel_size)
        else:
            #Carga multiples mascaras y las asocia con sus respectivos fotogramas
            masks = {}
            for f in os.listdir(IMAGE_MASK_OUT):
                if f.startswith(filename_main):
                    img = cv2.imread(os.path.join(IMAGE_MASK_OUT, f))
                    frame_num = int(re.search(r'\d+$', f[:-9]).group())
                    masks[frame_num] = img[:,:,0]

            # Realiza retoques en fotogramas utilizando multiples mascaras
            frames_inpainted = frame_inpainting(frames, masks, default_mask=0, kernel_size=inpainting_kernel_size)

        # Escribe fotogramas limpios en el disco
        for key, value in frames_inpainted.items():
            cv2.imwrite(CLEAN_IMAGE_OUT + filename_main + "_clean_frame" + str(key) + ".jpg", value)

        #Escribe los videos limpios en el disco
        generate_clean_video(frames_inpainted, os.path.join(CLEAN_VIDEO_OUT, filename_main + '_clean.avi'))
        
        '''    
        # Escribe videos limpios en el disco
        keys = list(frames_inpainted.keys())
        keys.sort()
        clean_vid_frames = [frames_inpainted[k] for k in keys]

        h, w, layers = clean_vid_frames[0].shape
        size = (w, h)

        out = cv2.VideoWriter(os.path.join(CLEAN_VIDEO_OUT + filename_main + '_clean.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
        for i in range(len(clean_vid_frames)):
            out.write(clean_vid_frames[i])
        out.release()  
        '''  
        
print("..............Proceso terminado :3 .........")

  0% (0 of 242) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Redimensionando la máscara 0 para que coincida con el fotograma.


  0% (1 of 242) |                        | Elapsed Time: 0:00:07 ETA:   0:31:22


Redimensionando la máscara 0 para que coincida con el fotograma.
Redimensionando la máscara 118 para que coincida con el fotograma.
Redimensionando la máscara 119 para que coincida con el fotograma.
Redimensionando la máscara 120 para que coincida con el fotograma.
Redimensionando la máscara 121 para que coincida con el fotograma.
Redimensionando la máscara 122 para que coincida con el fotograma.
Redimensionando la máscara 123 para que coincida con el fotograma.
Redimensionando la máscara 124 para que coincida con el fotograma.
Redimensionando la máscara 125 para que coincida con el fotograma.
Redimensionando la máscara 126 para que coincida con el fotograma.
Redimensionando la máscara 127 para que coincida con el fotograma.
Redimensionando la máscara 128 para que coincida con el fotograma.
Redimensionando la máscara 129 para que coincida con el fotograma.
Redimensionando la máscara 130 para que coincida con el fotograma.
Redimensionando la máscara 134 para que coincida con el fotogram

  0% (2 of 242) |                        | Elapsed Time: 0:00:16 ETA:   0:33:30


...

Redimensionando la máscara 0 para que coincida con el fotograma.


  4% (10 of 242) |                       | Elapsed Time: 0:01:46 ETA:   0:41:07


Redimensionando la máscara 0 para que coincida con el fotograma.


  4% (11 of 242) |#                      | Elapsed Time: 0:01:51 ETA:   0:38:57


KeyboardInterrupt: 